In [49]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd
from ra_dec import radec2dir, qv2dir, dir2radec, radec_diff
import asteroid_integrate
from asteroid_data import make_data_one_file, get_earth_pos

### Observation of Earth and Mars according to JPL

In [2]:
def add_col_planet(df):
    """Add a new column to a DataFrame with the MJD"""
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Order columns
    columns = ['mjd', 'JulianDate', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'LT', 'RG', 'RR']
    df = df[columns]
    return df

In [3]:
# Load the JPL position of Earth as CSV
df_earth = pd.read_csv('../data/jpl/testing/vectors-earth.txt', index_col=False)
# Add column for mjd
df_earth = add_col_planet(df_earth)

# Display the dataframe
df_earth

,mjd,JulianDate,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197,2455197.5,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55198,2455198.5,-0.196939,0.967049,-0.000017,-0.017145,-0.003447,9.036109e-07,0.005700,0.986899,0.000044
2,55199,2455199.5,-0.214053,0.963453,-0.000016,-0.017083,-0.003745,8.653246e-07,0.005700,0.986945,0.000049
3,55200,2455200.5,-0.231103,0.959559,-0.000015,-0.017017,-0.004042,7.855759e-07,0.005700,0.986997,0.000054
4,55201,2455201.5,-0.248085,0.955369,-0.000014,-0.016945,-0.004339,6.725245e-07,0.005701,0.987054,0.000059
...,...,...,...,...,...,...,...,...,...,...,...
3648,58845,2458845.5,-0.100787,0.986067,-0.000022,-0.017416,-0.001767,9.809367e-07,0.005725,0.991204,0.000013
3649,58846,2458846.5,-0.118187,0.984147,-0.000021,-0.017382,-0.002073,9.327289e-07,0.005725,0.991218,0.000015
3650,58847,2458847.5,-0.135550,0.981922,-0.000020,-0.017343,-0.002377,8.590659e-07,0.005725,0.991234,0.000017
3651,58848,2458848.5,-0.152871,0.979393,-0.000019,-0.017298,-0.002681,7.650397e-07,0.005725,0.991251,0.000019


In [4]:
# Load the JPL position of Mars as CSV
df_mars = pd.read_csv('../data/jpl/testing/vectors-mars.txt', index_col=False)
# Add column for mjd
df_mars = add_col_planet(df_mars)

# Display the dataframe
df_mars

,mjd,JulianDate,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197,2455197.5,-0.733418,1.457212,0.048394,-0.011980,-0.005093,0.000188,0.009426,1.632088,0.000842
1,55198,2455198.5,-0.745374,1.452070,0.048580,-0.011930,-0.005192,0.000184,0.009431,1.632927,0.000834
2,55199,2455199.5,-0.757278,1.446828,0.048763,-0.011879,-0.005291,0.000181,0.009436,1.633756,0.000826
3,55200,2455200.5,-0.769131,1.441488,0.048942,-0.011827,-0.005390,0.000178,0.009441,1.634578,0.000817
4,55201,2455201.5,-0.780931,1.436049,0.049118,-0.011774,-0.005488,0.000174,0.009445,1.635391,0.000809
...,...,...,...,...,...,...,...,...,...,...,...
3648,58845,2458845.5,-1.356374,-0.836106,0.015533,0.007920,-0.010678,-0.000418,0.009203,1.593445,-0.001143
3649,58846,2458846.5,-1.348404,-0.846754,0.015114,0.008019,-0.010616,-0.000419,0.009196,1.592299,-0.001149
3650,58847,2458847.5,-1.340336,-0.857339,0.014695,0.008117,-0.010553,-0.000420,0.009190,1.591146,-0.001155
3651,58848,2458848.5,-1.332170,-0.867860,0.014274,0.008215,-0.010490,-0.000421,0.009183,1.589988,-0.001161


In [5]:
def add_col_obs(df):
    """Add a new column to a DataFrame with the MJD"""
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Order columns
    columns = ['mjd', 'JulianDate', 'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',]
    df = df[columns]
    return df

In [6]:
# Load the JPL RA and DEC for Mars as fixed width file
df_obs = pd.read_fwf('../data/jpl/testing/observe-mars-earth-geocenter.txt')
# Add column for mjd
df_obs = add_col_obs(df_obs)

# Display the dataframe
df_obs

,mjd,JulianDate,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,55197,2455197.5,-0.749289,0.658994,0.065524,142.327061,18.799029,142.475968,18.752304,0.738832,-8.970408,6.144676
1,55198,2455198.5,-0.747395,0.661070,0.066233,142.179197,18.888379,142.328371,18.841711,0.733722,-8.724067,6.102178
2,55199,2455199.5,-0.745339,0.663317,0.066933,142.017468,18.981461,142.166897,18.934867,0.728756,-8.470333,6.060882
3,55200,2455200.5,-0.743120,0.665733,0.067625,141.841890,19.078173,141.991560,19.031671,0.723940,-8.208951,6.020822
4,55201,2455201.5,-0.740736,0.668315,0.068307,141.652504,19.178403,141.802402,19.132004,0.719276,-7.939695,5.982036
...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845,2458845.5,-0.567439,-0.823385,0.007031,233.190100,-18.727810,233.467326,18.791714,2.212868,12.191696,8.403866
3649,58846,2458846.5,-0.557754,-0.829978,0.006864,233.876730,-18.895847,234.154585,18.958738,2.205809,12.255895,8.345152
3650,58847,2458847.5,-0.547988,-0.836459,0.006694,234.565048,-19.061459,234.843518,19.123323,2.198712,12.318649,8.286133
3651,58848,2458848.5,-0.538142,-0.842829,0.006524,235.255049,-19.224614,235.534120,19.285436,2.191580,12.379967,8.226816


In [7]:
# Extract position and velocity of earth from df_earth
q_earth_jpl = np.array([df_earth.X.values, df_earth.Y.values, df_earth.Z.values]) * au
v_earth_jpl = np.array([df_earth.VX.values, df_earth.VY.values, df_earth.VZ.values]) * au / day

# Extract position of mars from df_mars
q_mars_jpl = np.array([df_mars.X.values, df_mars.Y.values, df_mars.Z.values]) * au
v_mars_jpl = np.array([df_mars.VX.values, df_mars.VY.values, df_mars.VZ.values]) * au / day

# Extract obstime_jd, ra, and dec from DataFrame
obstime_jd = df_obs.JulianDate.values
ra_jpl = df_obs.RA.values * deg
dec_jpl = df_obs.DEC.values * deg

# Vector of observation times in MJD format
obstime_mjd = jd_to_mjd(obstime_jd)

### Position & Observation of Earth and Mars according to Skyfield

In [8]:
# Manually load planetary positions using de435
planets_sf = load('../data/jpl/ephemeris/de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts = load.timescale()

# Generate vector of observation times in Skyfield format
obstime_sf = ts.tt_jd(obstime_jd)

In [9]:
# Observe mars from earth with Skyfield
obs_sf = earth_sf.at(obstime_sf).observe(mars_sf)

# Build Skyfield angle arrays (RA, DEC) and distance array (delta)
ra_sf_aa, dec_sf_aa, delta_sf_da = obs_sf.radec()

# Extract degrees and AU to get arrays of astropy angles and distances
ra_sf = ra_sf_aa._degrees * deg
dec_sf = dec_sf_aa._degrees * deg
delta_sf = delta_sf_da.au * au

In [10]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth_sf = earth_sf.at(obstime_sf).ecliptic_position().au * au
v_earth_sf = earth_sf.at(obstime_sf).ecliptic_velocity().km_per_s * km / second

# Mars
q_mars_sf = mars_sf.at(obstime_sf).ecliptic_position().au * au
v_mars_sf = mars_sf.at(obstime_sf).ecliptic_velocity().km_per_s * km / second

In [11]:
# Demonstrate that q_earth_sf is the same as q_earth_jpl
q_earth_eps = np.mean(np.linalg.norm(q_earth_sf - q_earth_jpl, axis=0))
v_earth_eps = np.mean(np.linalg.norm(v_earth_sf - v_earth_jpl, axis=0))
q_mars_eps = np.mean(np.linalg.norm(q_mars_sf - q_mars_jpl, axis=0))
v_mars_eps = np.mean(np.linalg.norm(v_mars_sf - v_mars_jpl, axis=0))

# Report
print('Difference between Skyfield (JPL ephem) and Horizons download:')
print(f'q_earth : {q_earth_eps:5.3e} au')
print(f'v_earth : {v_earth_eps:5.3e} au / day')
print(f'q_mars  : {q_mars_eps:5.3e} au')
print(f'v_mars  : {v_mars_eps:5.3e} au / day')

Difference between Skyfield (JPL ephem) and Horizons download:
q_earth : 1.406e-09 au
v_earth : 1.626e-08 au / day
q_mars  : 1.565e-09 au
v_mars  : 3.758e-08 au / day


In [12]:
# Alias q_earth, v_earth, q_mars, v_mars to the Skyfield interpolation for legibility
q_earth = q_earth_sf
v_earth = v_earth_sf
q_mars = q_mars_sf
v_mars = v_mars_sf

### Compare Skyfield vs JPL

In [13]:
# Compute difference in angles
diff_sf = radec_diff('JPL', 'Skyfield', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_sf, dec2=dec_sf, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_sf)
diff_median = np.median(diff_sf)
diff_max = np.max(diff_sf)

# Report results
print(f'Mean Angle Difference: JPL vs. Skyfield')
print(f'Mean  : {diff_mean:5.3f} seconds')
print(f'Median: {diff_median:5.3f} seconds')
print(f'Max   : {diff_max:5.3f} seconds')

Mean Angle Difference: JPL vs. Skyfield
Mean  : 1.598 seconds
Median: 1.822 seconds
Max   : 2.213 seconds


In [14]:
# Compute direction from Earth to Mars using the RA/DEC from JPL
u_e2m_jpl = radec2dir(ra_jpl, dec_jpl, obstime_mjd)

# Compute direction from Earth to Mars using the RA/DEC from Skyfield
u_e2m_sf = radec2dir(ra_sf, dec_sf, obstime_mjd)

# Difference in directions
u_e2m_diff = u_e2m_sf - u_e2m_jpl

# Norm of difference
u_diff_norm = np.linalg.norm(u_e2m_diff, axis=0)

# Mean
u_diff_mean = np.mean(u_diff_norm)
print(f'Mean norm of direction difference: JPL vs. Skyfield')
print(f'Mean : {u_diff_mean:5.2e}')

Mean norm of direction difference: JPL vs. Skyfield
Mean : 7.75e-06


### Calculate Direction Earth to Mars with qv2dir() and JPL Position / Velocity

In [15]:
# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_jpl = radec2dir(ra=ra_jpl, dec=dec_jpl, obstime_mjd=obstime_mjd)

In [16]:
# calculate direction with JPL data
u_mse = qv2dir(q_body=q_mars_jpl, v_body=v_mars_jpl, q_earth=q_earth_jpl)

In [17]:
# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

mean error:    1.604 arc seconds


### Calculate RA/DEC of Mars with MSE Function dir2radec() and JPL Directions

In [18]:
ra_mse, dec_mse = dir2radec(u_jpl, obstime_mjd)

In [19]:
# Compute difference in angles
diff_mse = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_sf, dec2=dec_sf, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_sf)
diff_median = np.median(diff_sf)
diff_max = np.max(diff_sf)

# Report results
print(f'Mean Angle Difference: JPL vs. MSE from JPL positions')
print(f'Mean  : {diff_mean:5.3f} seconds')
print(f'Median: {diff_median:5.3f} seconds')
print(f'Max   : {diff_max:5.3f} seconds')

Mean Angle Difference: JPL vs. MSE from JPL positions
Mean  : 1.598 seconds
Median: 1.822 seconds
Max   : 2.213 seconds


### Vectors of First 16 Asteroids from JPL

In [21]:
def add_cols_ast_vector(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number and mjd"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'LT', 'RG', 'RR']
    df = df[columns]
    return df

In [22]:
# List of dataframes; one per asteroid
df_ast_list = []

# Load the JPL position of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_ast_j = pd.read_csv(f'../data/jpl/testing/vectors-asteroid-{j:03d}.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_ast_j = add_cols_ast_vector(df=df_ast_j, asteroid_num=j)
    # Add this to list of frames
    df_ast_list.append(df_ast_j)

# Concatenate dataframes
df_ast = pd.concat(df_ast_list)

In [23]:
 df_ast

,asteroid_num,mjd,JulianDate,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,1,55197,2455197.5,-1.660333,-2.123236,0.238962,0.007615,-0.007150,-0.001627,0.015628,2.705909,0.000794
1,1,55198,2455198.5,-1.652706,-2.130370,0.237334,0.007640,-0.007118,-0.001630,0.015633,2.706703,0.000794
2,1,55199,2455199.5,-1.645053,-2.137472,0.235702,0.007665,-0.007086,-0.001634,0.015637,2.707497,0.000795
3,1,55200,2455200.5,-1.637376,-2.144542,0.234066,0.007689,-0.007054,-0.001637,0.015642,2.708293,0.000796
4,1,55201,2455201.5,-1.629675,-2.151580,0.232427,0.007713,-0.007022,-0.001641,0.015646,2.709088,0.000796
...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845,2458845.5,2.517677,-0.513079,-0.043698,0.001590,0.011210,-0.000568,0.014842,2.569797,-0.000671
3649,16,58846,2458846.5,2.519245,-0.501865,-0.044266,0.001546,0.011219,-0.000568,0.014838,2.569129,-0.000665
3650,16,58847,2458847.5,2.520770,-0.490641,-0.044834,0.001503,0.011228,-0.000567,0.014834,2.568466,-0.000660
3651,16,58848,2458848.5,2.522251,-0.479409,-0.045400,0.001459,0.011236,-0.000566,0.014830,2.567809,-0.000655


### Observations of First 16 Asteroids from JPL

In [29]:
def add_cols_ast_observation(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number, mjd, and direction"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5).astype(np.int32)
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',]
    df = df[columns]
    return df

In [30]:
# List of dataframes; one per asteroid
df_obs_list = []

# Load the JPL observations of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_obs_j = pd.read_fwf(f'../data/jpl/testing/observer-asteroid-{j:03d}-earth-geocenter.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_obs_j = add_cols_ast_observation(df=df_obs_j, asteroid_num=j)
    # Add this to list of frames
    df_obs_list.append(df_obs_j)

# Concatenate dataframes
df_obs = pd.concat(df_obs_list)

In [31]:
df_obs

,asteroid_num,mjd,JulianDate,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197,2455197.5,-0.430702,-0.899812,0.069523,243.215442,-17.105913,243.358581,-17.131844,3.437877,-12.468091,28.591952
1,1,55198,2455198.5,-0.424384,-0.902835,0.069195,243.625145,-17.196033,243.768548,-17.221645,3.430618,-12.668422,28.531584
2,1,55199,2455199.5,-0.418063,-0.905804,0.068867,244.034084,-17.284935,244.177730,-17.310227,3.423244,-12.868277,28.470254
3,1,55200,2455200.5,-0.411741,-0.908720,0.068539,244.442231,-17.372621,244.586099,-17.397589,3.415755,-13.067723,28.407966
4,1,55201,2455201.5,-0.405417,-0.911583,0.068211,244.849560,-17.459094,244.993632,-17.483728,3.408150,-13.266749,28.344720
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845,2458845.5,0.867713,-0.496854,-0.014471,332.583400,-12.175911,332.842722,-12.079671,3.017646,17.406609,25.096998
3649,16,58846,2458846.5,0.871109,-0.490873,-0.014611,332.963279,-12.044021,333.222363,-11.947459,3.027660,17.268707,25.180280
3650,16,58847,2458847.5,0.874478,-0.484841,-0.014750,333.344527,-11.910963,333.603364,-11.814086,3.037593,17.128990,25.262894
3651,16,58848,2458848.5,0.877821,-0.478758,-0.014889,333.727108,-11.776749,333.985688,-11.679564,3.047446,16.987592,25.344833


### Calculate Asteroid Direction with qv2dir() and JPL Position / Velocity

In [264]:
# arrays of asteroid position & velocity from JPL
q_ast_jpl = np.array([df_ast.X.values, df_ast.Y.values, df_ast.Z.values]) * au
v_ast_jpl = np.array([df_ast.VX.values, df_ast.VY.values, df_ast.VZ.values]) * au / day
mjd_ast = df_ast.mjd.values

# tile earth position to match shape of q_ast
q_earth_jpl_tile = np.tile(q_earth_jpl, 16)

# arrays of asteroid angles from JPL
ra_ast_jpl = df_obs.RA.values * deg
dec_ast_jpl = df_obs.DEC.values * deg
obstime_mjd = df_obs.mjd.values.astype(np.float64)

# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_jpl = radec2dir(ra=ra_ast_jpl, dec=dec_ast_jpl, obstime_mjd=obstime_mjd)

# calculate direction with JPL data
u_mse = qv2dir(q_body=q_ast_jpl, v_body=v_ast_jpl, q_earth=q_earth_jpl_tile)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

mean error:    0.873 arc seconds


### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [66]:
ast_elt = asteroid_integrate.load_data()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [151]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [140]:
ast_elt

,asteroid_num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [152]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth.mjd.size
N_row = N_ast * N_t

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

Shape of data frames df_ast and df_obs:
N_ast =    16 asteroids
N_t   =  3653 observation times
N_row = 58448 rows in df_ast and df_obs


In [153]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [243]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]

# for some reason i don't understant, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

(16, 3653, 3)

In [276]:
# Get position of Earth using utility function
q_earth_mse = get_earth_pos(obstime_mjd).transpose()

# tile earth position to match shape of q_ast
q_earth_mse_tile = np.tile(q_earth_mse, 16)

q_earth_mse.shape

(3, 58448)

In [245]:
# shape JPL positions to match q_ast_mse with three axes (asteroid_num, time_idx, space_dim)
q_ast_jpl_3d = np.zeros((N_ast, N_t, 3))
q_ast_jpl_3d[:, :, 0] = df_ast.X.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 1] = df_ast.Y.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 2] = df_ast.Z.values.reshape((N_ast, N_t))
q_ast_jpl_3d.shape

(16, 3653, 3)

In [246]:
# verify that coordinates are close (bit different b/c MSE calculations heliocentric vs. barycentric for JPL)
q_diff = q_ast_mse_3d - q_ast_jpl_3d
q_diff_norm = np.linalg.norm(q_diff, axis=2)
np.mean(q_diff_norm)

0.0045478332141220676

In [256]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))

In [257]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

In [258]:
q_ast_jpl.shape

(3, 58448)

In [254]:
q_ast_jpl.shape

(3, 58448)

In [273]:
# Compute direction from MSE position and velocity
u_mse = qv2dir(q_body=q_ast_mse, v_body=v_ast_mse, q_earth=q_earth_mse_tile)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

ValueError: operands could not be broadcast together with shapes (3,58448) (3,935168) 

In [265]:
v_ast_mse.shape

(3, 58448)

In [266]:
v_ast_jpl.shape

(3, 58448)

In [267]:
q_earth_jpl_tile.shape

(3, 58448)

In [268]:
q_earth_mse_tile.shape

(58448, 48)

In [269]:
q_earth_jpl.shape

(3, 3653)

In [275]:
q_earth_mse_tile.shape

(3, 935168)